In [ ]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect recipe results.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


In [ ]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [ ]:
mlflow.set_tracking_uri("file:////content/drive/MyDrive/mlruns_new")

In [ ]:
mlflow.set_experiment("logistic")

<Experiment: artifact_location='file://content/drive/MyDrive/mlruns_new/835408792279618950', creation_time=1730716931717, experiment_id='835408792279618950', last_update_time=1730716931717, lifecycle_stage='active', name='logistic', tags={}>

In [ ]:
df = pd.read_csv("/content/obesity_data_with_status_50000.csv")
df.head()

,Age,Gender,Height_cm,Weight_kg,BMI,Physical_Activity_Level,Diet_Type,Smoking_Habits,Alcohol_Consumption,Family_History_Obesity,Blood_Pressure,Cholesterol_Levels,Education_Level,Income_Level,Geographical_Region,Obesity_Status
0,56,0,172.005936,68.686179,23.215738,4,1,2,0,1,Hypertension Stage 2,2,1,3,2,Normal weight
1,69,1,161.678497,79.937723,30.580688,2,0,0,0,0,Elevated,1,2,1,3,Obese
2,46,1,172.078597,99.707918,33.672545,4,1,1,1,0,Normal,3,2,3,2,Obese
3,32,1,176.845709,76.314302,24.401520,1,2,0,1,0,Normal,3,1,2,1,Normal weight
4,60,1,170.875161,84.397860,28.905039,2,1,0,2,1,Elevated,2,3,3,2,Overweight


In [ ]:
df.shape

(50000, 16)

In [ ]:
label_encoder = LabelEncoder()
df['Obesity_Status'] = label_encoder.fit_transform(df['Obesity_Status'])
df['Blood_Pressure'] = label_encoder.fit_transform(df['Blood_Pressure'])

In [ ]:
def remove_outliers_zscore(df, column_names, threshold=3):
    z_scores = np.abs((df[column_names] - df[column_names].mean()) / df[column_names].std())
    filtered_df = df[(z_scores < threshold).all(axis=1)]
    return filtered_df

continuous_columns = ['Height_cm', 'Weight_kg']
df_cleaned = remove_outliers_zscore(df, continuous_columns)

print(f"Data shape after outlier removal: {df_cleaned.shape}")

Data shape after outlier removal: (49744, 16)


In [ ]:
scaler = StandardScaler()
df_cleaned[['Height_cm', 'Weight_kg']] = scaler.fit_transform(df_cleaned[['Height_cm', 'Weight_kg']])

In [ ]:
class_counts = df_cleaned['Obesity_Status'].value_counts()

scale_factor = class_counts.max() / class_counts

def augment_data(df, target_col, scale_factor):
    frames = []
    for label, factor in scale_factor.items():
        df_class = df[df[target_col] == label]
        df_class_replicated = pd.concat([df_class] * int(np.round(factor)), ignore_index=True)
        frames.append(df_class_replicated)

    df_augmented = pd.concat(frames).sample(frac=1).reset_index(drop=True)  # Shuffle the data
    return df_augmented

df_augmented = augment_data(df_cleaned, 'Obesity_Status', scale_factor)
print(f"New class distribution after augmentation:\n{df_augmented['Obesity_Status'].value_counts()}")

New class distribution after augmentation:
Obesity_Status
3    18308
0    17442
2    15094
1    12631
Name: count, dtype: int64


In [ ]:
Q1 = df_augmented['Obesity_Status'].quantile(0.25)
Q3 = df_augmented['Obesity_Status'].quantile(0.75)
IQR = Q3 - Q1


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_augmented[(df_augmented['Obesity_Status'] < lower_bound) | (df_augmented['Obesity_Status'] > upper_bound)]

print("Outliers in Obesity_Status column:", len(outliers))

Outliers in Obesity_Status column: 0


In [ ]:
X = df_augmented.drop('Obesity_Status', axis=1)
y = df_augmented['Obesity_Status']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 15% validation, 15% test

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

Training set shape: (44432, 15)
Validation set shape: (9521, 15)
Test set shape: (9522, 15)


In [ ]:
with mlflow.start_run(run_name="Run5"):
    # Logistic Regression Model
    log_reg = LogisticRegression(max_iter=500, solver='sag', class_weight='balanced')
    log_reg.fit(X_train, y_train)
    y_pred_log_reg = log_reg.predict(X_val)

    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred_log_reg)
    print(f"Logistic Regression Accuracy: {accuracy}")
    class_report = classification_report(y_val, y_pred_log_reg, output_dict=True)
    cm = confusion_matrix(y_val, y_pred_log_reg)

    # Log parameters and metrics
    mlflow.log_param("model_type", "Logistic Regression")
    mlflow.log_param("max_iter", log_reg.max_iter)
    mlflow.log_param("solver", log_reg.solver)
    mlflow.log_param("class_weight", "balanced")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision_weighted", class_report['weighted avg']['precision'])
    mlflow.log_metric("recall_weighted", class_report['weighted avg']['recall'])
    mlflow.log_metric("f1_weighted", class_report['weighted avg']['f1-score'])

    mlflow.sklearn.log_model(log_reg, "logistic_regression_model")

Logistic Regression Accuracy: 0.987921436823863


2024/11/04 10:48:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
mlflow.set_experiment("SVM")

2024/11/04 11:05:27 INFO mlflow.tracking.fluent: Experiment with name 'SVM' does not exist. Creating a new experiment.


<Experiment: artifact_location='file://content/drive/MyDrive/mlruns_new/454708959125817374', creation_time=1730718327881, experiment_id='454708959125817374', last_update_time=1730718327881, lifecycle_stage='active', name='SVM', tags={}>

In [ ]:
# Number of runs
num_runs = 5

# Initialize a list of hyperparameter settings to test manually
# You can modify these parameter values as needed
parameter_settings = [
    {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'},
    {'C': 1, 'gamma': 'scale', 'kernel': 'linear'},
    {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'},
    {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'},
    {'C': 0.5, 'gamma': 'scale', 'kernel': 'linear'}
]

# Loop through the parameter settings
for run, params in enumerate(parameter_settings):
    with mlflow.start_run(run_name=f"SVM_Experiment_Run_{run + 1}"):
        # Initialize the SVM model with the specified parameters
        svm = SVC(C=params['C'], gamma=params['gamma'], kernel=params['kernel'], class_weight='balanced')

        # Fit the model
        svm.fit(X_train, y_train)

        # Make predictions
        y_pred_svm = svm.predict(X_val)

        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred_svm)
        print(f"SVM Run {run + 1} Accuracy: {accuracy}")
        class_report = classification_report(y_val, y_pred_svm, output_dict=True)
        cm = confusion_matrix(y_val, y_pred_svm)

        # Log parameters and metrics
        mlflow.log_param("model_type", "SVM")
        mlflow.log_param("C", params['C'])
        mlflow.log_param("gamma", params['gamma'])
        mlflow.log_param("kernel", params['kernel'])
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision_weighted", class_report['weighted avg']['precision'])
        mlflow.log_metric("recall_weighted", class_report['weighted avg']['recall'])
        mlflow.log_metric("f1_weighted", class_report['weighted avg']['f1-score'])

        # Log the SVM model
        mlflow.sklearn.log_model(svm, "svm_model")

SVM Run 1 Accuracy: 0.9951685747295452


2024/11/04 11:20:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVM Run 2 Accuracy: 0.997689318348913


2024/11/04 11:20:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVM Run 3 Accuracy: 0.9887616846969857


2024/11/04 11:20:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVM Run 4 Accuracy: 0.9841403213948114


2024/11/04 11:21:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVM Run 5 Accuracy: 0.9974792563806323


2024/11/04 11:21:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
mlflow.set_experiment("RandomForestClassifier")

2024/11/04 14:19:44 INFO mlflow.tracking.fluent: Experiment with name 'RandomForestClassifier' does not exist. Creating a new experiment.


<Experiment: artifact_location='file://content/drive/MyDrive/mlruns_new/844163624781780033', creation_time=1730729984971, experiment_id='844163624781780033', last_update_time=1730729984971, lifecycle_stage='active', name='RandomForestClassifier', tags={}>

In [ ]:
# Number of runs
num_runs = 5

# Initialize a list of hyperparameter settings to test manually
# You can modify these parameter values as needed
parameter_settings = [
    {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2},
    {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 5},
    {'n_estimators': 150, 'max_depth': 20, 'min_samples_split': 10},
    {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 2},
    {'n_estimators': 75, 'max_depth': 15, 'min_samples_split': 3}
]

# Loop through the parameter settings
for run, params in enumerate(parameter_settings):
    with mlflow.start_run(run_name=f"RandomForest_Experiment_Run_{run + 1}"):
        # Initialize the Random Forest model with the specified parameters
        rf = RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            class_weight='balanced',
            random_state=42
        )

        # Fit the model
        rf.fit(X_train, y_train)

        # Make predictions
        y_pred_rf = rf.predict(X_val)

        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred_rf)
        print(f"Random Forest Run {run + 1} Accuracy: {accuracy}")
        class_report = classification_report(y_val, y_pred_rf, output_dict=True)
        cm = confusion_matrix(y_val, y_pred_rf)

        # Log parameters and metrics
        mlflow.log_param("model_type", "Random Forest")
        mlflow.log_param("n_estimators", params['n_estimators'])
        mlflow.log_param("max_depth", params['max_depth'])
        mlflow.log_param("min_samples_split", params['min_samples_split'])
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision_weighted", class_report['weighted avg']['precision'])
        mlflow.log_metric("recall_weighted", class_report['weighted avg']['recall'])
        mlflow.log_metric("f1_weighted", class_report['weighted avg']['f1-score'])

        # Log the Random Forest model
        mlflow.sklearn.log_model(rf, "random_forest_model")

Random Forest Run 1 Accuracy: 1.0


2024/11/04 14:20:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest Run 2 Accuracy: 1.0


2024/11/04 14:20:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest Run 3 Accuracy: 1.0


2024/11/04 14:20:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest Run 4 Accuracy: 1.0


2024/11/04 14:20:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest Run 5 Accuracy: 1.0


2024/11/04 14:20:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
mlflow.set_experiment("XGBoost")

2024/11/04 14:24:39 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost' does not exist. Creating a new experiment.


<Experiment: artifact_location='file://content/drive/MyDrive/mlruns_new/706479386585321452', creation_time=1730730279578, experiment_id='706479386585321452', last_update_time=1730730279578, lifecycle_stage='active', name='XGBoost', tags={}>

In [ ]:
# Number of runs
num_runs = 5

parameter_settings = [
    {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 3},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 5},
    {'n_estimators': 150, 'learning_rate': 0.01, 'max_depth': 7},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 4},
    {'n_estimators': 100, 'learning_rate': 0.2, 'max_depth': 6}
]

# Loop through the parameter settings
for run, params in enumerate(parameter_settings):
    with mlflow.start_run(run_name=f"XGBoost_Experiment_Run_{run + 1}"):
        # Initialize the XGBoost model with the specified parameters
        xgb = XGBClassifier(
            n_estimators=params['n_estimators'],
            learning_rate=params['learning_rate'],
            max_depth=params['max_depth'],
            use_label_encoder=False,
            eval_metric='logloss',  # Suppress warning with `use_label_encoder=False`
            random_state=42
        )

        # Fit the model
        xgb.fit(X_train, y_train)

        # Make predictions
        y_pred_xgb = xgb.predict(X_val)

        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred_xgb)
        print(f"XGBoost Run {run + 1} Accuracy: {accuracy}")
        class_report = classification_report(y_val, y_pred_xgb, output_dict=True)
        cm = confusion_matrix(y_val, y_pred_xgb)

        # Log parameters and metrics
        mlflow.log_param("model_type", "XGBoost")
        mlflow.log_param("n_estimators", params['n_estimators'])
        mlflow.log_param("learning_rate", params['learning_rate'])
        mlflow.log_param("max_depth", params['max_depth'])
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision_weighted", class_report['weighted avg']['precision'])
        mlflow.log_metric("recall_weighted", class_report['weighted avg']['recall'])
        mlflow.log_metric("f1_weighted", class_report['weighted avg']['f1-score'])

        # Log the XGBoost model
        mlflow.sklearn.log_model(xgb, "xgboost_model")

XGBoost Run 1 Accuracy: 0.9974792563806323


2024/11/04 14:24:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


XGBoost Run 2 Accuracy: 0.9982144732696145


2024/11/04 14:24:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


XGBoost Run 3 Accuracy: 0.9983195042537548


2024/11/04 14:25:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


XGBoost Run 4 Accuracy: 0.9983195042537548


2024/11/04 14:25:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


XGBoost Run 5 Accuracy: 0.9981094422854743


2024/11/04 14:25:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
mlflow.set_experiment("TabNetClassifier")

2024/11/06 08:39:33 INFO mlflow.tracking.fluent: Experiment with name 'TabNetClassifier' does not exist. Creating a new experiment.


<Experiment: artifact_location='file://content/drive/MyDrive/mlruns_new/401100004534551629', creation_time=1730882373244, experiment_id='401100004534551629', last_update_time=1730882373244, lifecycle_stage='active', name='TabNetClassifier', tags={}>

In [ ]:
# Assuming X_train, y_train, X_val, and y_val are in pandas DataFrame or NumPy array format

# Convert data to NumPy arrays if not already
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_val_np = np.array(X_val)
y_val_np = np.array(y_val)

# Label encode target variable if it's categorical
label_encoder = LabelEncoder()
y_train_np = label_encoder.fit_transform(y_train_np)
y_val_np = label_encoder.transform(y_val_np)

In [ ]:
num_runs = 5

parameter_settings = [
    {'n_d': 8, 'n_a': 8, 'n_steps': 3, 'gamma': 1.3, 'lambda_sparse': 0.0001},
    {'n_d': 16, 'n_a': 16, 'n_steps': 5, 'gamma': 1.5, 'lambda_sparse': 0.001},
    {'n_d': 24, 'n_a': 24, 'n_steps': 7, 'gamma': 1.7, 'lambda_sparse': 0.005},
    {'n_d': 32, 'n_a': 32, 'n_steps': 3, 'gamma': 1.4, 'lambda_sparse': 0.01},
    {'n_d': 16, 'n_a': 16, 'n_steps': 4, 'gamma': 1.3, 'lambda_sparse': 0.0005}
]

for run, params in enumerate(parameter_settings):
    with mlflow.start_run(run_name=f"TabNet_Experiment_Run_{run + 1}"):
        # Initialize the TabNet model with the specified parameters
        tabnet = TabNetClassifier(
            n_d=params['n_d'],
            n_a=params['n_a'],
            n_steps=params['n_steps'],
            gamma=params['gamma'],
            lambda_sparse=params['lambda_sparse'],
            seed=42
        )

        # Fit the model
        tabnet.fit(
            X_train_np, y_train_np,
            eval_set=[(X_val_np, y_val_np)],
            eval_metric=['accuracy'],
            max_epochs=100,
            patience=10,
            batch_size=256,
            virtual_batch_size=128
        )

        # Make predictions
        y_pred_tabnet = tabnet.predict(X_val_np)

        # Calculate metrics
        accuracy = accuracy_score(y_val_np, y_pred_tabnet)
        print(f"TabNet Run {run + 1} Accuracy: {accuracy}")
        class_report = classification_report(y_val_np, y_pred_tabnet, output_dict=True)
        cm = confusion_matrix(y_val_np, y_pred_tabnet)

        # Log parameters and metrics
        mlflow.log_param("model_type", "TabNet")
        mlflow.log_param("n_d", params['n_d'])
        mlflow.log_param("n_a", params['n_a'])
        mlflow.log_param("n_steps", params['n_steps'])
        mlflow.log_param("gamma", params['gamma'])
        mlflow.log_param("lambda_sparse", params['lambda_sparse'])
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision_weighted", class_report['weighted avg']['precision'])
        mlflow.log_metric("recall_weighted", class_report['weighted avg']['recall'])
        mlflow.log_metric("f1_weighted", class_report['weighted avg']['f1-score'])

        # Log the TabNet model
        mlflow.sklearn.log_model(tabnet, "tabnet_model")

epoch 0  | loss: 0.34807 | val_0_accuracy: 0.39135 |  0:00:06s
epoch 1  | loss: 0.18196 | val_0_accuracy: 0.97332 |  0:00:10s
epoch 2  | loss: 0.1842  | val_0_accuracy: 0.952   |  0:00:16s
epoch 3  | loss: 0.16192 | val_0_accuracy: 0.97721 |  0:00:22s
epoch 4  | loss: 0.17604 | val_0_accuracy: 0.96891 |  0:00:26s
epoch 5  | loss: 0.15868 | val_0_accuracy: 0.94906 |  0:00:32s
epoch 6  | loss: 0.16701 | val_0_accuracy: 0.93005 |  0:00:38s
epoch 7  | loss: 0.15174 | val_0_accuracy: 0.95295 |  0:00:42s
epoch 8  | loss: 0.14751 | val_0_accuracy: 0.9729  |  0:00:49s
epoch 9  | loss: 0.14622 | val_0_accuracy: 0.97479 |  0:00:55s
epoch 10 | loss: 0.16057 | val_0_accuracy: 0.97647 |  0:01:01s
epoch 11 | loss: 0.15406 | val_0_accuracy: 0.97868 |  0:01:07s
epoch 12 | loss: 0.14495 | val_0_accuracy: 0.97595 |  0:01:12s
epoch 13 | loss: 0.14627 | val_0_accuracy: 0.94391 |  0:01:18s
epoch 14 | loss: 0.15169 | val_0_accuracy: 0.98551 |  0:01:23s
epoch 15 | loss: 0.14535 | val_0_accuracy: 0.9854  |  0

2024/11/06 08:43:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


epoch 0  | loss: 0.38922 | val_0_accuracy: 0.40752 |  0:00:07s
epoch 1  | loss: 0.18442 | val_0_accuracy: 0.85831 |  0:00:18s
epoch 2  | loss: 0.18244 | val_0_accuracy: 0.97364 |  0:00:28s
epoch 3  | loss: 0.17587 | val_0_accuracy: 0.97542 |  0:00:37s
epoch 4  | loss: 0.15782 | val_0_accuracy: 0.91745 |  0:00:46s
epoch 5  | loss: 0.16074 | val_0_accuracy: 0.97679 |  0:00:55s
epoch 6  | loss: 0.15274 | val_0_accuracy: 0.93509 |  0:01:04s
epoch 7  | loss: 0.16042 | val_0_accuracy: 0.94738 |  0:01:12s
epoch 8  | loss: 0.15186 | val_0_accuracy: 0.96135 |  0:01:21s
epoch 9  | loss: 0.15126 | val_0_accuracy: 0.96723 |  0:01:29s
epoch 10 | loss: 0.14129 | val_0_accuracy: 0.95011 |  0:01:38s
epoch 11 | loss: 0.14439 | val_0_accuracy: 0.96103 |  0:01:47s
epoch 12 | loss: 0.15084 | val_0_accuracy: 0.96292 |  0:01:54s
epoch 13 | loss: 0.15496 | val_0_accuracy: 0.97196 |  0:02:03s
epoch 14 | loss: 0.15242 | val_0_accuracy: 0.96397 |  0:02:11s
epoch 15 | loss: 0.15938 | val_0_accuracy: 0.96555 |  0

2024/11/06 08:46:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


epoch 0  | loss: 0.62576 | val_0_accuracy: 0.44229 |  0:00:16s
epoch 1  | loss: 0.27643 | val_0_accuracy: 0.87984 |  0:00:32s
epoch 2  | loss: 0.21881 | val_0_accuracy: 0.96639 |  0:00:48s
epoch 3  | loss: 0.19705 | val_0_accuracy: 0.93824 |  0:01:04s
epoch 4  | loss: 0.17508 | val_0_accuracy: 0.97385 |  0:01:21s
epoch 5  | loss: 0.17005 | val_0_accuracy: 0.96534 |  0:01:37s
epoch 6  | loss: 0.16277 | val_0_accuracy: 0.96471 |  0:01:54s
epoch 7  | loss: 0.16691 | val_0_accuracy: 0.98425 |  0:02:10s
epoch 8  | loss: 0.15445 | val_0_accuracy: 0.96565 |  0:02:24s
epoch 9  | loss: 0.15442 | val_0_accuracy: 0.94917 |  0:02:36s
epoch 10 | loss: 0.16169 | val_0_accuracy: 0.93971 |  0:02:48s
epoch 11 | loss: 0.15527 | val_0_accuracy: 0.98109 |  0:03:02s
epoch 12 | loss: 0.16239 | val_0_accuracy: 0.95547 |  0:03:15s
epoch 13 | loss: 0.14551 | val_0_accuracy: 0.97595 |  0:03:27s
epoch 14 | loss: 0.14683 | val_0_accuracy: 0.954   |  0:03:40s
epoch 15 | loss: 0.13719 | val_0_accuracy: 0.97091 |  0

2024/11/06 08:50:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


epoch 0  | loss: 0.34556 | val_0_accuracy: 0.41172 |  0:00:07s
epoch 1  | loss: 0.1728  | val_0_accuracy: 0.88573 |  0:00:14s
epoch 2  | loss: 0.15894 | val_0_accuracy: 0.94454 |  0:00:21s
epoch 3  | loss: 0.15342 | val_0_accuracy: 0.98099 |  0:00:28s
epoch 4  | loss: 0.15323 | val_0_accuracy: 0.97899 |  0:00:36s
epoch 5  | loss: 0.15281 | val_0_accuracy: 0.9123  |  0:00:45s
epoch 6  | loss: 0.14643 | val_0_accuracy: 0.98383 |  0:00:53s
epoch 7  | loss: 0.14572 | val_0_accuracy: 0.96902 |  0:01:00s
epoch 8  | loss: 0.14621 | val_0_accuracy: 0.95505 |  0:01:07s
epoch 9  | loss: 0.14894 | val_0_accuracy: 0.96723 |  0:01:14s
epoch 10 | loss: 0.14892 | val_0_accuracy: 0.97941 |  0:01:21s
epoch 11 | loss: 0.13988 | val_0_accuracy: 0.96933 |  0:01:29s
epoch 12 | loss: 0.13786 | val_0_accuracy: 0.98067 |  0:01:35s
epoch 13 | loss: 0.14574 | val_0_accuracy: 0.94318 |  0:01:43s
epoch 14 | loss: 0.13913 | val_0_accuracy: 0.94339 |  0:01:50s
epoch 15 | loss: 0.13744 | val_0_accuracy: 0.97584 |  0

2024/11/06 08:52:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


epoch 0  | loss: 0.41929 | val_0_accuracy: 0.38389 |  0:00:08s
epoch 1  | loss: 0.18578 | val_0_accuracy: 0.93278 |  0:00:14s
epoch 2  | loss: 0.18115 | val_0_accuracy: 0.92921 |  0:00:22s
epoch 3  | loss: 0.17531 | val_0_accuracy: 0.95652 |  0:00:29s
epoch 4  | loss: 0.16409 | val_0_accuracy: 0.96765 |  0:00:37s
epoch 5  | loss: 0.14944 | val_0_accuracy: 0.96271 |  0:00:43s
epoch 6  | loss: 0.16722 | val_0_accuracy: 0.97637 |  0:00:51s
epoch 7  | loss: 0.15541 | val_0_accuracy: 0.94444 |  0:00:58s
epoch 8  | loss: 0.16078 | val_0_accuracy: 0.96912 |  0:01:08s
epoch 9  | loss: 0.1483  | val_0_accuracy: 0.9458  |  0:01:15s
epoch 10 | loss: 0.15088 | val_0_accuracy: 0.97406 |  0:01:23s
epoch 11 | loss: 0.15544 | val_0_accuracy: 0.94538 |  0:01:30s
epoch 12 | loss: 0.16065 | val_0_accuracy: 0.97322 |  0:01:38s
epoch 13 | loss: 0.15392 | val_0_accuracy: 0.93772 |  0:01:44s
epoch 14 | loss: 0.16448 | val_0_accuracy: 0.96944 |  0:01:52s
epoch 15 | loss: 0.16991 | val_0_accuracy: 0.97185 |  0

2024/11/06 08:56:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
